In [1]:
# importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 


In [2]:
# loading the datasets
advertisers = pd.read_csv('advertisers.csv')
locations = pd.read_csv('locations.csv')
results = pd.read_csv('results.csv')

In [3]:
# Displaying the basic information about the datasets
print("Advertisers Dataset Info:")
print(advertisers.info())
print("\nLocations Dataset Info:")
print(locations.info())
print("\nResults Dataset Info:")
print(results.info())

Advertisers Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20832 entries, 0 to 20831
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Page ID                   20832 non-null  int64 
 1   Page name                 20832 non-null  object
 2   Disclaimer                20832 non-null  object
 3   Amount spent (INR)        20832 non-null  object
 4   Number of ads in Library  20832 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 813.9+ KB
None

Locations Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Location name       36 non-null     object
 1   Amount spent (INR)  36 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 708.0+ bytes
None

Results Dataset Info:
<class 'pandas.core.frame.Dat

# 1. Data Cleaning & Preparation

In [7]:
# standardizing and cleaning the column names
locations['Location name'] = locations['Location name'].str.strip().str.title()
results['State'] = results['State'].str.strip().str.title()

# No 'Revenue' or 'Cost' columns in results, so nothing to convert here


In [12]:
# indentifying the columns for the datasets
advertisers_columns = advertisers.columns.tolist()
locations_columns = locations.columns.tolist()
results_columns = results.columns.tolist()

# Displaying the columns of each dataset
advertisers_columns


['Page ID',
 'Page name',
 'Disclaimer',
 'Amount spent (INR)',
 'Number of ads in Library']

In [10]:
locations_columns


['Location name', 'Amount spent (INR)']

In [11]:
results_columns

['_id',
 'Sl No',
 'State',
 'PC_Name',
 'Total Electors',
 'Polled (%)',
 'Total Votes',
 'Phase']

# 2. Explanatory Analysis

In [ ]:
# merging results with locations on 'State'
locations.head()


,Location name,Amount spent (INR)
0,Andaman And Nicobar Islands,377858
1,Andhra Pradesh,100819732
2,Arunachal Pradesh,1385654
3,Assam,17478091
4,Bihar,53619242


In [21]:
results.head()


,_id,Sl No,State,PC_Name,Total Electors,Polled (%),Total Votes,Phase
0,1,1.0,Andaman & Nicobar Islands,Andaman & Nicobar Islands,315148,64.10,202018,1.0
1,2,2.0,Arunachal Pradesh,Arunachal East,375310,83.31,312658,1.0
2,3,3.0,Arunachal Pradesh,Arunachal West,517384,73.60,380783,1.0
3,4,4.0,Assam,Dibrugarh,1659588,76.75,1273744,1.0
4,5,5.0,Assam,Jorhat,1727121,79.89,1379749,1.0


In [22]:
advertisers.head()

,Page ID,Page name,Disclaimer,Amount spent (INR),Number of ads in Library
0,121439954563203,Bharatiya Janata Party (BJP),Bharatiya Janata Party (BJP),193854342,43455
1,351616078284404,Indian National Congress,Indian National Congress,108787100,846
2,132715103269897,Ama Chinha Sankha Chinha,Ama Chinha Sankha Chinha,73361399,1799
3,192856493908290,Ama Chinha Sankha Chinha,Ama Chinha Sankha Chinha,32294327,680
4,109470364774303,Ellorum Nammudan,Populus Empowerment Network Private Limited,22399499,879


In [24]:
# merging the datasets
locations['Location name'] = locations['Location name'].str.strip().str.lower()
results['State'] = results['State'].str.strip().str.lower()
merged_data = results.merge(locations, left_on='State', right_on='Location name', how='left')

In [25]:
# viewing the merged data
merged_data.head()

,_id,Sl No,State,PC_Name,Total Electors,Polled (%),Total Votes,Phase,Location name,Amount spent (INR)
0,1,1.0,andaman & nicobar islands,Andaman & Nicobar Islands,315148,64.10,202018,1.0,NaN,NaN
1,2,2.0,arunachal pradesh,Arunachal East,375310,83.31,312658,1.0,arunachal pradesh,1385654.0
2,3,3.0,arunachal pradesh,Arunachal West,517384,73.60,380783,1.0,arunachal pradesh,1385654.0
3,4,4.0,assam,Dibrugarh,1659588,76.75,1273744,1.0,assam,17478091.0
4,5,5.0,assam,Jorhat,1727121,79.89,1379749,1.0,assam,17478091.0


In [28]:
# total ad spend by state
state_ad_spend = merged_data.groupby('State')['Amount spent (INR)'].sum().reset_index()

In [36]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"

state_ad_spend = merged_data.groupby('State')['Amount spent (INR)'].sum().reset_index()

fig = px.bar(state_ad_spend, x='State', y='Amount spent (INR)',
             labels={'State': 'State', 'Amount spent (INR)': 'Ad Spend (INR)'},
             title='Total Ad Spend by State')

fig.update_layout(xaxis={'categoryorder': 'total descending'},
                  xaxis_tickangle=-90,
                  width=800,
                  height=600)

fig.show()

In [37]:
# Average voter turnout by state
avg_turnout_by_state = merged_data.groupby('State')['Polled (%)'].mean().reset_index()

In [38]:
# visualizing the total ad spend by state
fig = px.bar(avg_turnout_by_state, x = 'State', y = 'Polled (%)',
             labels = {'State': 'State', 'Polled (%)': 'Average Voter Turnout (%)'},
             title = 'Average Voter Turnout by State')
fig.update_layout(xaxis={'categoryorder': 'total descending'},
                  xaxis_tickangle=-90,
                  width=800,
                  height=600)

In [45]:
# Top 5 parties by ad spend
advertisers['Amount spent (INR)']=pd.to_numeric(advertisers['Amount spent (INR)'], errors='coerce')
advertisers.dropna(subset=['Amount spent (INR)'], inplace=True)
party_ad_spend = advertisers.groupby('Page name')['Amount spent (INR)'].sum().sort_values(ascending=False)
top_5_parties = party_ad_spend.head(5).reset_index()
colors = ['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']

fig = px.pie(top_5_parties, names='Page name', values='Amount spent (INR)',
                title='Top 5 Parties by Ad Spend',
                color_discrete_sequence=colors,
                labels={'Page name': 'Party', 'Amount spent (INR)': 'Ad Spend (INR)'})
fig.update_traces( textinfo='percent')
fig.update_layout(showlegend=True,
                  legend=dict(orientation='v', yanchor='top', y=1, xanchor='left', x=-0.3),
                  title=dict(x=0.5, y=0.95, xanchor='center', yanchor='top'),
                  margin=dict(l=200, r=50, t=100, b=50)
)
fig.show()


In [49]:
# correlation between ad spend and voter turnout
correlation_data = merged_data[['Amount spent (INR)', 'Polled (%)']].corr()
correlation_data

,Amount spent (INR),Polled (%)
Amount spent (INR),1.000000,-0.010688
Polled (%),-0.010688,1.000000


In [52]:
# Relationship between ad spend and voter turnout by parliamentary constituency
merged_constituency_data = results.merge(locations, left_on='State', right_on='Location name', how='left')
fig = px.scatter(merged_constituency_data, x='Amount spent (INR)', y='Polled (%)',
                 color='State', hover_name='Location name',
                 title='Ad Spend vs Voter Turnout by Parliamentary Constituency',
                 opacity=0.8,
                 labels={'Amount spent (INR)': 'Ad Spend (INR)', 'Polled (%)': 'Voter Turnout (%)'})
fig.update_layout(width=800, height=600)
fig.show()


In [55]:
# Distribution of ad spending 
fig = px.histogram(merged_data,  x='Amount spent (INR)', nbins=30,marginal='box',
                 title='Distribution of Ad Spending',
                 labels={'Amount spent (INR)': 'Ad Spend (INR)'})
fig.update_layout(bargap=0.1, width=800, height=600)
fig.update_traces(marker=dict(line=dict(color='black', width=1)))
fig.show()

In [61]:
# Analyzing ad spend and voter turn out by election phase
import plotly.graph_objects as go

phase_analysis = merged_data.groupby('Phase').agg({'Amount spent (INR)': 'sum', 'Polled (%)': 'mean'}).reset_index()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=phase_analysis['Phase'],
    y=phase_analysis['Amount spent (INR)'],
    name='Ad Spend (INR)',
    marker_color='indianred',
    yaxis='y1'
))
fig.add_trace(go.Scatter(
    x=phase_analysis['Phase'],
    y=phase_analysis['Polled (%)'],
    name='Average Voter Turnout (%)',
    marker_color='lightsalmon',
    yaxis='y2'
))
fig.update_layout(
    title='Ad Spend and Voter Turnout by Election Phase',
    xaxis=dict(title='Election Phase'),
    yaxis=dict(
        title={'text': 'Ad Spend (INR)', 'font': {'color': 'indianred'}},
        tickfont=dict(color='lightsalmon')
    ),
    yaxis2=dict(
        title={'text': 'Average Voter Turnout (%)', 'font': {'color': 'indianred'}},
        tickfont=dict(color='lightsalmon'),
        overlaying='y',
        side='right'
    ),
    legend=dict(x=0.1, y=1.1, orientation='h'),
    width=800,
    height=600
)
fig.show()
